<a href="https://colab.research.google.com/github/mt508/machine-learning-/blob/main/heartattack_predict_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [123]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Activation, LeakyReLU, ELU

from tensorflow.keras import  layers, models
import matplotlib.pyplot as plt
from zlib import crc32
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

In [2]:
data=pd.read_csv("/content/Medicaldataset.csv")

In [3]:
data=data.drop(columns=["CK-MB","Troponin"])

In [219]:
import numpy as np
import pandas as pd
from zlib import crc32

# Example dataset
# data = pd.read_csv("your_file.csv")

# Make sure data is a DataFrame
print(type(data))

# Function definitions
def is_id_in_test_set(identifier, test_ratio):
    return crc32(np.int64(identifier)) < test_ratio * 2**32

def split_data_with_id(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: is_id_in_test_set(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]

# Create artificial stable ID
data["id"] = np.arange(len(data))

# Make a copy (optional)
data_with_id = data.copy()

# Train-test split
train_set, test_set = split_data_with_id(data_with_id, 0.2, "id")

print(train_set.head())
print(test_set.head())


<class 'pandas.core.frame.DataFrame'>
   Age  Gender  Heart rate  Systolic blood pressure  Diastolic blood pressure  \
0   64       1          66                      160                        83   
1   21       1          94                       98                        46   
3   64       1          70                      120                        55   
4   55       1          64                      112                        65   
6   32       0          40                      179                        68   

   Blood sugar    Result  id  
0        160.0  negative   0  
1        296.0  positive   1  
3        270.0  positive   3  
4        300.0  negative   4  
6        102.0  negative   6  
    Age  Gender  Heart rate  Systolic blood pressure  \
2    55       1          64                      160   
5    58       0          61                      112   
12   64       1          60                      199   
16   86       0          73                      114   
23   30  

In [218]:
train_copy=train_set.copy()
X = train_copy.drop(columns=["Result","id"])
# Target
y = data["Result"]

# Encode target
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y)


In [220]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled_train = scaler.fit_transform(X)


In [222]:
import numpy as np
unique, counts = np.unique(y_train, return_counts=True)
dict(zip(unique, counts))


{'negative': np.int64(407), 'positive': np.int64(648)}

In [205]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, ELU
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import AdamW

model5 = Sequential([
    Dense(64, input_shape=(6,)),
    ELU(),

    Dense(128),
    BatchNormalization(),
    ELU(),

    Dense(64),
    ELU(),

    Dropout(0.3),

    Dense(1, activation='sigmoid')
])

model5.compile(
    optimizer=AdamW(
        learning_rate=5e-4,
        weight_decay=0.001
    ),
    loss='binary_crossentropy',
    metrics=['accuracy', Precision(name='precision')]
)


Epoch 1/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.5272 - loss: 0.7586 - precision: 0.6344 - val_accuracy: 0.4906 - val_loss: 0.7033 - val_precision: 0.5660 - learning_rate: 5.0000e-04
Epoch 2/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4665 - loss: 0.7790 - precision: 0.5858 - val_accuracy: 0.5189 - val_loss: 0.7082 - val_precision: 0.5962 - learning_rate: 5.0000e-04
Epoch 3/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5322 - loss: 0.7198 - precision: 0.6362 - val_accuracy: 0.4811 - val_loss: 0.7072 - val_precision: 0.5600 - learning_rate: 5.0000e-04
Epoch 4/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5099 - loss: 0.7336 - precision: 0.6151 - val_accuracy: 0.5377 - val_loss: 0.7012 - val_precision: 0.6111 - learning_rate: 5.0000e-04
Epoch 5/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5472 - loss: 0.7199 - precision: 0.6300 - val_accuracy: 0.4906 - val_loss: 0.7019 - val_precision: 0.5660 - learning_rate: 5.0000e-04
Epoc

In [208]:
print(X_scaled_train.shape)
print(y_train.shape)


(1055, 6)
(1319,)


In [223]:
# After splitting
train_copy = train_set.copy()
test_copy  = test_set.copy()

# Features
X_train = train_copy.drop(columns=["Result", "id"])
X_test  = test_copy.drop(columns=["Result", "id"])

# Target
y_train = train_copy["Result"]
y_test  = test_copy["Result"]

# Encode target
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train_ = le.fit_transform(y_train)
y_test_  = le.transform(y_test)

# Scale features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled_train_ = scaler.fit_transform(X_train)
X_scaled_test_  = scaler.transform(X_test)

print(X_scaled_train.shape, y_train.shape)
print(X_scaled_test.shape, y_test.shape)


(1055, 6) (1055,)
(264, 6) (264,)


In [228]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall

model_ = Sequential([
    Dense(64, activation="tanh", input_shape=(6,), kernel_regularizer=l2(0.001)),
    BatchNormalization(),

    Dense(32, activation="tanh", kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.25),

    Dense(16, activation="tanh"),

    Dense(1, activation="sigmoid")
])

model_.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy", Precision(name="precision"), Recall(name="recall")]
)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [229]:
from sklearn.utils import class_weight
import numpy as np

cw = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(y_train),
    y=y_train
)

cw_dict = dict(enumerate(cw))
print("Class weights:", cw_dict)


Class weights: {0: np.float64(1.296068796068796), 1: np.float64(0.8140432098765432)}


In [230]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

es = EarlyStopping(
    monitor='val_loss',
    patience=12,
    restore_best_weights=True
)

lr = ReduceLROnPlateau(
    monitor='val_loss',
    patience=4,
    factor=0.5,
    min_lr=1e-6
)

history = model_.fit(
    X_scaled_train_, y_train_,
    validation_split=0.15,
    epochs=200,
    batch_size=32,
    class_weight=cw_dict,
    callbacks=[es, lr],
    verbose=1
)


Epoch 1/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - accuracy: 0.5562 - loss: 0.7614 - precision: 0.6593 - recall: 0.5575 - val_accuracy: 0.5849 - val_loss: 0.7130 - val_precision: 0.6923 - val_recall: 0.5625 - learning_rate: 0.0010
Epoch 2/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5907 - loss: 0.7235 - precision: 0.6901 - recall: 0.6069 - val_accuracy: 0.5786 - val_loss: 0.7113 - val_precision: 0.6883 - val_recall: 0.5521 - learning_rate: 0.0010
Epoch 3/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5878 - loss: 0.7102 - precision: 0.7070 - recall: 0.5835 - val_accuracy: 0.6038 - val_loss: 0.7092 - val_precision: 0.7324 - val_recall: 0.5417 - learning_rate: 0.0010
Epoch 4/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6076 - loss: 0.7134 - precision: 0.7216 - recall: 0.5884 - val_accuracy: 0.6164 - val_loss: 0.7027 - val_precision: 0.7397 - val_recall: 0.5625 - learning_rate: 0.0010
Epoch 5/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.58